### Introduction

In this project, number of missing value imputation methods were experimented and the result is evaluated through Area Under the Curve (AUC).
In general, missing value imputation methods can be divided into machine learning method and statistical method.
Statistical method are much easy to implement while Machine Learning methods, as suggested by its name, required knowledge in ML, therefore much complicated.

The dataset employed is California Housing Prices provided by Cam Nugent on Kaggle

link to dataset: https://www.kaggle.com/camnugent/california-housing-prices

this project will be divided into 2 part:
1. data are removed randomly from 1 columns
2. data are removed non-randomly from 1 columns

In each part of the project, both statistical method and ML methods of imputation will be implemented and evaluated.
Total_bedrooms were selected as the variable to be tested

---
---
### Preparation

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score

In [2]:
# cnames = ['age','workclass','fnlwgt','education','education-num',
#           'marital-status','occupation','relationship','race','sex',
#          'capital-gain','capital-loss','hours-per-week','native-country']
df = pd.read_csv('housing.csv', index_col = False)
df.dropna(inplace=True)
df.reset_index(inplace=True,drop=True)
def string_conversion_for_op(x):
    if x == '<1H OCEAN':
        return 1
    if x == 'INLAND':
        return 2
    if x == 'ISLAND':
        return 3
    if x == 'NEAR BAY':
        return 4
    if x == 'NEAR OCEAN':
        return 5
df['ocean_proximity'] = df['ocean_proximity'].apply(string_conversion_for_op)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,4
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,4
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,4
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,4
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,4


#### Remove Data Randomly

In [3]:
#Identifying 5% data count
20433 * (5/100)

1021.6500000000001

In [4]:
#Create random number generator
random_values = []
for _ in range(0,5000):
    x = np.random.randint(0,20433)
    if x not in random_values:
        random_values.append(x) 
    if len(random_values) == 1022:
        break
random_values[0:10]

[6006, 3888, 14844, 11031, 1067, 1412, 7301, 8959, 9211, 14905]

In [5]:
len(random_values)
min(random_values)

11

In [6]:
# save the value into a list and remove it from the dataframe
random_list_original_values = []
for row_index in random_values:
    random_list_original_values.append(df.iloc[row_index,4])
for row_index in random_values:
    df.iloc[row_index,4] = np.nan

In [7]:
random_df = df
random_df.head(4)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,4
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,4
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,4
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,4


#### Remove data non-randomly

after some data review, housing_median_age is selected as the target variable,  1220 entry of total_bedrooms will be removed from the entry with housing_median_age of 20 and lower, simulating a real world scenario where missing values are due to certain causes instead of purely random.

In [8]:
#### Recalling original df

df = pd.read_csv('housing.csv', index_col = False)
df.dropna(inplace=True)
df.reset_index(inplace=True,drop=True)
df['ocean_proximity'] = df['ocean_proximity'].apply(string_conversion_for_op)

In [9]:
len(df[df['housing_median_age'] < 20])

5774

In [10]:
#### among 5774 samples that satisfy the condition, 1220 entries are removed in the following codes
non_random_lst = df[df['housing_median_age'] < 20].index
selected_lst = random.sample(list(non_random_lst),1022)
len(selected_lst)

1022

In [11]:
# Saving Original lst and removing the values
nonrandom_list_original_values = []
for row_index in selected_lst:
    nonrandom_list_original_values.append(df.iloc[row_index,4])
for row_index in selected_lst:
    df.iloc[row_index,4] = np.nan

print(len(df[df['total_bedrooms'].isnull()]))
nonrandom_df = df

1022


In [12]:
nonrandom_df.head(4)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,4
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,4
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,4
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,4


In [13]:
#### fail-safe checking, the total entries must be 20433, total answer list must be 1022
print(len(random_df),len(nonrandom_df))
print(len(random_list_original_values),len(nonrandom_list_original_values))

20433 20433
1022 1022


---
---
### Main Coding
#### Random missing values imputation
##### simple imputation, mean filling

In [14]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
fitted_random_df = imp.fit_transform(random_df['total_bedrooms'].values.reshape(-1,1))

In [15]:
len(fitted_random_df.flatten())

20433

##### complex imputation, 1d knn filling

In [16]:
knnimp = KNNImputer()
knn_random_df = knnimp.fit_transform(random_df['total_bedrooms'].values.reshape(-1,1))
len(knn_random_df)

20433

multiple d knn filling

In [17]:
knnimp2 = KNNImputer()
knn2_random_df = knnimp.fit_transform(random_df)

In [18]:
knn2_random_df = pd.DataFrame(knn2_random_df)
knn2_random_df

,0,1,2,3,4,5,6,7,8,9
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,4.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,4.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,4.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,4.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,4.0
...,...,...,...,...,...,...,...,...,...,...
20428,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,2.0
20429,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,2.0
20430,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,2.0
20431,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,2.0


In [19]:
random_df['filled_total_bedrooms'] = fitted_random_df
random_df['knn_total_bedrooms'] = knn_random_df
random_df['knn2_total_bedrooms'] = knn2_random_df.iloc[:,4]

In [20]:
random_df.isna().any()

longitude                False
latitude                 False
housing_median_age       False
total_rooms              False
total_bedrooms            True
population               False
households               False
median_income            False
median_house_value       False
ocean_proximity          False
filled_total_bedrooms    False
knn_total_bedrooms       False
knn2_total_bedrooms      False
dtype: bool

In [21]:
#combine all interested data into 1 new datarfame for evaluation
final_df = random_df.iloc[:,-3:]
df = pd.read_csv('housing.csv', index_col = False)
df.dropna(inplace=True)
df.reset_index(inplace=True,drop=True)
df['ocean_proximity'] = df['ocean_proximity'].apply(string_conversion_for_op)
final_df['ori_total_bedrooms'] = df['total_bedrooms']
final_df.isna().any()

filled_total_bedrooms    False
knn_total_bedrooms       False
knn2_total_bedrooms      False
ori_total_bedrooms       False
dtype: bool

In [22]:
#identify data for auc
random_final_df = final_df.iloc[random_values,:]

In [23]:
random_final_df

,filled_total_bedrooms,knn_total_bedrooms,knn2_total_bedrooms,ori_total_bedrooms
6006,538.090207,538.090207,579.4,497.0
3888,538.090207,538.090207,195.4,144.0
14844,538.090207,538.090207,176.4,150.0
11031,538.090207,538.090207,277.8,215.0
1067,538.090207,538.090207,449.4,378.0
...,...,...,...,...
2400,538.090207,538.090207,256.8,226.0
7233,538.090207,538.090207,158.4,83.0
12503,538.090207,538.090207,3417.8,5027.0
11793,538.090207,538.090207,836.6,901.0


i did some experiment to test how knnimputer works when only 1 feature was given, turns out it works exactly as mean value imputation.

In [24]:
print('1. the r2 for mean filling method is:', r2_score(random_final_df.iloc[:,3].values,random_final_df.iloc[:,0].values))
print('2. the r2 for knn 1d filling method is:', r2_score(random_final_df.iloc[:,3].values,random_final_df.iloc[:,1].values))
print('3. the r2 for knn md filling method is:', r2_score(random_final_df.iloc[:,3].values,random_final_df.iloc[:,2].values))

1. the r2 for mean filling method is: -0.00010750660630609588
2. the r2 for knn 1d filling method is: -0.00010750660630609588
3. the r2 for knn md filling method is: 0.8749564405280993


---
#### Non-Random missing value imputation

In [25]:
nonrandom_df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,4
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,4
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,4
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,4
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,4
...,...,...,...,...,...,...,...,...,...,...
20428,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,2
20429,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,2
20430,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,2
20431,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,2


In [26]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
fitted_nonrandom_df = imp.fit_transform(nonrandom_df['total_bedrooms'].values.reshape(-1,1))
knnimp2 = KNNImputer()
knn2_nonrandom_df = knnimp.fit_transform(nonrandom_df)
knn2_nonrandom_df = pd.DataFrame(knn2_nonrandom_df)

In [27]:
nonrandom_df['filled_total_bedrooms'] = fitted_nonrandom_df
nonrandom_df['knn2_total_bedrooms'] = knn2_nonrandom_df.iloc[:,4]
nonrandom_df.shape

(20433, 12)

In [28]:
df = pd.read_csv('housing.csv', index_col = False)
df.dropna(inplace=True)
df.reset_index(inplace=True,drop=True)
df['ocean_proximity'] = df['ocean_proximity'].apply(string_conversion_for_op)
nonrandom_final_df = df.iloc[:,]
nonrandom_final_df['ori_total_bedrooms'] = df['total_bedrooms']

In [34]:
nonrandom_final_df = nonrandom_df.iloc[:,-2:]
nonrandom_final_df['ori_total_bedrooms'] = df['total_bedrooms']
nonrandom_final_df = nonrandom_final_df.iloc[non_random_lst,:]

In [35]:
nonrandom_final_df

,filled_total_bedrooms,knn2_total_bedrooms,ori_total_bedrooms
59,43.000000,43.0,43.0
75,462.000000,462.0,462.0
77,243.000000,243.0,243.0
80,289.000000,289.0,289.0
87,348.000000,348.0,348.0
...,...,...,...
20425,527.706352,393.4,416.0
20429,150.000000,150.0,150.0
20430,485.000000,485.0,485.0
20431,409.000000,409.0,409.0


In [36]:
print('1. the r2 for mean filling method is:', r2_score(nonrandom_final_df.iloc[:,2].values,nonrandom_final_df.iloc[:,0].values))
print('3. the r2 for knn md filling method is:', r2_score(nonrandom_final_df.iloc[:,2].values,nonrandom_final_df.iloc[:,1].values))

1. the r2 for mean filling method is: 0.7986804942943196
3. the r2 for knn md filling method is: 0.9816980010407128


---
---
#### Result and Conclusion
The result suggested ML method of missing value imputation is a much better alternative to statistical mean value imputation. Among all results, KNN method scores even highly in non-random circumstances, suggesting when the missing value is known to associated with certain variable, ml method seems to work very well, a 0.98 of r2 was acquired. However, please note that there are more statistical method that employs method with greater complexity may reduce the gap.